# Load multiple COGs into xarray dataset

In [ ]:
import satsearch
from satstac import Items
import geopandas as gpd
import xarray as xr

In [ ]:
gfa = gpd.read_file('aoi.geojson')
gfa

In [ ]:
bbox = gfa.bounds.values[0]
west, north, east, south = bbox

In [ ]:
properties =  ["landsat:tier=T1"] 

bbox = (west, south, east, north) #(min lon, min lat, max lon, max lat)

results = satsearch.Search.search(collection='landsat-8-l1', 
                        bbox=bbox, 
                        datetime='2019-07-15/2019-07-31',
                        sort=['<datetime'], #earliest scene first
                        property=properties)
print('%s items' % results.found())

In [ ]:
results.found()

In [ ]:
import holoviews as hv
import hvplot.xarray
import hvplot.pandas
import geoviews as gv

In [ ]:
items = results.items()
items.save('items-landsat8.json')
items = Items.load('items-landsat8.json')
#items.bbox()

In [ ]:
gfl = gpd.read_file('items-landsat8.json')
gfl = gfl.sort_values('datetime').reset_index(drop=True)
print('records:', len(gfl))
gfl.head()

In [ ]:
# Plot search AOI and frames on a map

# just keep id for hover tips
cols = gfl.loc[:,('id','geometry')]

footprints = cols.hvplot(geo=True, line_color='k', alpha=0.1, title='Landsat 8 T1')
aoi = gfa.hvplot(geo=True, line_color='b', fill_color=None)
tiles = gv.tile_sources.CartoEco.options(width=700, height=500) #.redim.range(Latitude=(45, 50), Longitude=(-126,-120)) 
labels = gv.tile_sources.StamenLabels.options(level='annotation')
tiles * footprints * aoi * labels

In [ ]:
# Print URLS's we'll work with
urls = [items[x].asset('nir')['href'] for x in range(4)]

In [ ]:
# Load these into an xarray dataset
# NIR band URLS of these 4 images
da1 = xr.open_rasterio(urls[0], chunks={'band': 1, 'x': 1024, 'y': 1024})
da2 = xr.open_rasterio(urls[1], chunks={'band': 1, 'x': 1024, 'y': 1024})
da3 = xr.open_rasterio(urls[2], chunks={'band': 1, 'x': 1024, 'y': 1024})
da4 = xr.open_rasterio(urls[3], chunks={'band': 1, 'x': 1024, 'y': 1024})

In [ ]:
# NOTE: per julia signells suggestino, want filename in attributes as well
da1.attrs['filename'] = items[0].id
da1.attrs['date'] = items[0].date.strftime('%Y-%m-%d')
da1

In [ ]:
da2.attrs['filename'] = items[1].id
da2.attrs['date'] = items[1].date.strftime('%Y-%m-%d')
da2

In [ ]:
da3.attrs['filename'] = items[2].id
da3.attrs['date'] = items[2].date.strftime('%Y-%m-%d')
da3

In [ ]:
da4.attrs['filename'] = items[3].id
da4.attrs['date'] = items[3].date.strftime('%Y-%m-%d')
da4

### NOTES: 

* these have the same CRS, but slightly different grids and dimensions

In [ ]:
# Visualize 1
da1.hvplot.image(rasterize=True, width=700, height=500, cmap='magma')

In [ ]:
#index = gpd.pd.DatetimeIndex(items.dates(), name='time')
datasets = [da1,da2,da3,da4]
da = xr.concat(datasets, dim='band')
print('Dataset size (Gb): ', da.nbytes/1e9)

In [ ]:
da

### NOTES:
* stacked data array, expanded coordinates to fit all the images
* lost attributes on individual datasets

In [ ]:
# Option 2, create a vrt pointing to urls, open the vrt
import os 
cmd = 'gdalbuildvrt mosaic.vrt /vsicurl/{} /vsicurl/{} /vsicurl/{} /vsicurl/{}'.format(*urls)
print(cmd)
os.system(cmd)

In [ ]:
!cat mosaic.vrt

In [ ]:
da = xr.open_rasterio('mosaic.vrt')
da

In [ ]:
### Notes: mosaicing happens here rather than stacking into a dataset


In [ ]:
# Option 2, create a time dimension, each COG goes into a DataSet Variable
